In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import copy
%matplotlib inline

In [4]:
df = pd.read_csv("train.csv")
df = df.rename(columns = {'Subject_Car_Colour': 'Car_Colour'})
df.head(10)
#df.Gender[df['Gender'].isin(['Male', 'Female'])].value_counts().to_dict()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1
5,ID_00D3EF6,2010-10-21,2011-10-20,Male,37,2010-10-21,2,NaN,NaN,NaN,NaN,NaN,Car Classic,0
6,ID_00HHZ8Y,2010-08-02,2011-08-01,Female,40,2010-08-02,3,NaN,NaN,NaN,Ikeja,Lagos,CVTP,0
7,ID_00M1X6U,2010-09-12,2011-09-11,Male,43,2010-09-12,1,Saloon,NaN,TOYOTA,NaN,NaN,CarSafe,0
8,ID_00N7XT1,2010-07-06,2011-07-05,Female,71,2010-07-06,2,JEEP,Black,REXTON,Badagry,Benue,Car Classic,0
9,ID_00QF156,2010-12-18,2011-12-17,Female,58,2010-12-18,1,NaN,NaN,NaN,Eti-Osa,Eti-Osa,Car Classic,0


## Calculate NaN Values in each Column

In [75]:
for col in df.columns:
    print(f"The number of NaN values in {col} is: \t {df[col].isna().sum()}")

The number of NaN values in ID is: 	 0
The number of NaN values in Policy Start Date is: 	 0
The number of NaN values in Policy End Date is: 	 0
The number of NaN values in Gender is: 	 359
The number of NaN values in Age is: 	 0
The number of NaN values in First Transaction Date is: 	 0
The number of NaN values in No_Pol is: 	 0
The number of NaN values in Car_Category is: 	 3738
The number of NaN values in Car_Colour is: 	 6962
The number of NaN values in Subject_Car_Make is: 	 2476
The number of NaN values in LGA_Name is: 	 6476
The number of NaN values in State is: 	 6488
The number of NaN values in ProductName is: 	 0
The number of NaN values in target is: 	 0


In [76]:
features = [col for col in df.columns if col != 'target']
X = df[features]
y = df['target']

In [77]:
df.iloc[0]

ID                         ID_0040R73
Policy Start Date          2010-05-14
Policy End Date            2011-05-13
Gender                           Male
Age                                30
First Transaction Date     2010-05-14
No_Pol                              1
Car_Category                   Saloon
Car_Colour                      Black
Subject_Car_Make               TOYOTA
LGA_Name                          NaN
State                             NaN
ProductName               Car Classic
target                              0
Name: 0, dtype: object

### Replacing NaN values in The Car_Colour  Column with a choosen Color using probability distributions of the Cars

In [79]:
Colours = dict(df.Car_Colour.value_counts())
Coloured = df.Car_Colour.notna().sum()
Colours = dict(map(lambda colour: (colour, Colours[colour] / Coloured), Colours.keys()))   # keys = Colours, values = Probability distribution of each car 

Colour_name = [key for key in Colours.keys()]
proba = [value for value in Colours.values()]

def distribute_Na(colour):
    if pd.isnull(colour):
        colour = "".join(random.choices(Colour_name, proba))
    return colour

'Gold'

In [82]:
df['Car_Colour'] = df['Car_Colour'].apply(lambda colour: distribute_Na(colour))


0

In [64]:
df.Car_Colour

0                Black
1                 Grey
2                  Red
3               [Grey]
4             [Silver]
             ...      
12074            Black
12075    [As Attached]
12076           [Blue]
12077            White
12078            Black
Name: Car_Colour, Length: 12079, dtype: object

In [25]:
df.Car_Colour.value_counts()

AttributeError: 'DataFrame' object has no attribute 'Car_Colour'

In [8]:
from sklearn.metrics import f1_score, classification_report

from sklearn.metrics import f1_score
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1', f1_score(y_true, y_hat), True

0        0
1        1
2        0
3        0
4        1
        ..
12074    1
12075    0
12076    0
12077    0
12078    0
Name: target, Length: 12079, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

for i in cat_col:
    le = LabelEncoder()
    le.fit(tr[i]);  tr[i] = le.transform(tr[i])